In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [2]:
model_list = ["Qwen/Qwen1.5-1.8B-Chat", "TinyLlama/TinyLlama-1.1B-Chat-v1.0"]

In [3]:
prompt_configs = [
    {
        "system": """You are a Bayesian modeling assistant specialized in probabilistic regression.
                      Your role is to estimate prior distributions over regression coefficients (weights)
                      in a Bayesian polynomial regression model.
                      Given a list of standardized input features, your job is to specify a Gaussian prior
                      (mean and standard deviation) over the weight of each feature.
                      These priors represent beliefs about whether a feature is likely to increase or decrease
                      the target value, and how confident we are about that belief.
                      """,
        "user": """ You are helping design a Bayesian linear regression model to predict the target variable 'MedHouseVal'
                    using standardized features. The inputs are features such as {features}, which have all been
                    z-score normalized (mean 0, std 1).

                    Your task is to guess a Gaussian prior (mean and standard deviation) over the **regression weight** for each feature.
                    The prior mean represents your belief about how the feature affects the target. dont gave same thing for everyone.
                    The standard deviation reflects how confident you are in that belief.

                    ⚠️ Important:
                    - These are **priors over weights**, not the features themselves.
                    - Do not use the original feature values to set these priors.
                    - All features are standardized, so weight magnitudes are expected to be small.
                    - Keep the prior means in the range **-1.0 to +1.0**, and standard deviations between **0.1 and 1.0**.

                    Please output only a JSON object with keys as feature names and mean and std.""",
        "features": ['MedInc', 'HouseAge', 'AveRooms','AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
    },
    {
        "system": """You are a Bayesian modeling assistant specialized in probabilistic neural networks.
                      Your role is to estimate prior distributions over the input weights
                      in a Bayesian neural network for binary classification.
                      Given a list of standardized input features, your job is to specify a Gaussian prior
                      (mean and standard deviation) over the input-layer weight of each feature.
                      These priors represent beliefs about how a feature is likely to affect
                      the probability of a positive class (malignant), and how confident we are about that belief.""",
        "user": """You are helping design a Bayesian neural network model to predict whether a tumor is malignant (1) or benign (0)
                    using standardized diagnostic features. The inputs are features such as {features}, which have all been
                    z-score normalized.

                    Your task is to guess a Gaussian prior (mean and standard deviation) over the **input-layer weight** for each feature.
                    The prior mean represents your belief about how the {features} influences the model’s confidence in a malignant diagnosis.
                    The standard deviation reflects how confident you are in that belief.

                    ⚠️ Important:
                    - These are **priors over input weights**, not the features themselves.
                    - Do not use the original feature values to set these priors.
                    - All features are standardized, so weight magnitudes are expected to be small but can be explored by high std values.
                    - Keep the prior non zero mean between the range **-1.0 to +1.0**, and standard deviation can be decided by you for each feature.
                    - We have to be a little careful here since the relationship between these specific features and the response is likely to be complex and nonlinear.

                    Please output only a JSON object with keys as feature names and values as dictionaries containing 'mean' and 'std' values.""",
        "features": ['mean radius', 'mean texture', 'mean perimeter', 'mean area',
                      'mean smoothness', 'mean compactness', 'mean concavity',
                      'mean concave points', 'mean symmetry', 'mean fractal dimension',
                      'radius error', 'texture error', 'perimeter error', 'area error',
                      'smoothness error', 'compactness error', 'concavity error',
                      'concave points error', 'symmetry error',
                      'fractal dimension error', 'worst radius', 'worst texture',
                      'worst perimeter', 'worst area', 'worst smoothness',
                      'worst compactness', 'worst concavity', 'worst concave points',
                      'worst symmetry', 'worst fractal dimension']
    },
    {
        "system" :"""You are a simulator of a logistic regression predictive model
                    for predicting whether the given customer will buy or not,i.e., online shopper purchasing intention.
                    Here the inputs are various online site data of the user and the output is
                    the probability of user purchasing from the site from online site data of the user.
                    Specifically, the target/label is ['revenue'] with mapping
                    ['notbuy' = 0, 'buy' = 1].
                    With your best guess, you can provide the probabilities of a user purchasing for the given user online data.""",
        "user":"""I am a data scientist with a dataset and the task: predicting user purchase on an online site from given user online site data.
                  I would like to use your model to predict the buying or not of my samples.
                  I have a dataset that is made up of the following features: {features} out of which some are categorical.
                  All of the feature values are standardized using the z-score. so the mean also should not be large because features are standardized.
                  By thinking about how each feature might be related to a purchase of
                  ['buy', 'notbuy'] , and whether each feature is positively or
                  negatively correlated with the outcome of
                  ['notbuy' = 0, 'buy' = 1],
                  I would like you to guess the
                  mean and standard deviation for a normal distribution prior for each feature
                  for a logistic regression model can be used for
                  predicting user purchase from user online site data characteristics.
                  Please respond with a JSON object with the feature names as keys
                  and a nested dictionary of mean and standard deviation as values.
                  But std values can be large if you are not sure
                  whilst a small standard deviation indicates that you are confident in your guess.
                  Please only respond with a JSON with mean and std for each feature, no other text.""",
        "features": ['Administrative', 'Administrative_Duration', 'Informational',
                    'Informational_Duration', 'ProductRelated', 'BounceRates', 'PageValues',
                    'SpecialDay', 'Month', 'OperatingSystems', 'Browser', 'Region',
                    'TrafficType', 'VisitorType', 'Weekend']
    },
      {
        "system": """You are a Bayesian modeling assistant specialized in probabilistic clustering.
                  Your role is to estimate prior distributions over feature relevance weights
                  in a Bayesian Gaussian Mixture Model (GMM) for unsupervised clustering.

                  Given a list of standardized input features, your job is to specify a Gaussian prior
                  (mean and standard deviation) over a weight associated with each feature.
                  These weights influence how strongly each feature contributes to separating clusters.
                  """,

        "user": """You are helping design a Bayesian Gaussian Mixture Model (GMM) to cluster data points
                based on the following standardized features: {features}.

                Your task is to guess a Gaussian prior (mean and standard deviation) over the **feature weight**
                for each input variable. These weights determine how relevant or irrelevant the feature is
                in distinguishing between latent clusters.

                ⚠️ Important:
                - These are **priors over weights**, not the features themselves.
                - All features are z-score normalized (mean 0, std 1).
                - Do not use original feature values or distributions to set priors.
                - The mean should reflect your belief in the direction and strength of the feature's cluster-separating influence.
                - The standard deviation reflects your uncertainty in that belief, it might be even large if you dont feel the condfidence.
                - Keep prior nonzero mean in the range **-1.0 to +1.0**,

                Please respond with a JSON object where keys are feature names and values are objects
                with `"mean"` and `"std"` with one value each""",

        "features": ["sepal_length", "sepal_width", "petal_length", "petal_width"]
      }
]

In [4]:
def run_model_on_prompts(model_id, prompt_configs):
    print(f"\n Loading model: {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    for i, config in enumerate(prompt_configs):
        feature_str = ", ".join(config["features"])
        user_prompt = config["user"].format(features=feature_str)

        messages = [
            {"role": "system", "content": config["system"]},
            {"role": "user", "content": user_prompt}
        ]

        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=1200)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"\n Prompt #{i+1} output:\n{result}\n" + "-"*60)

In [5]:
for model_id in model_list:
    run_model_on_prompts(model_id, prompt_configs)


 Loading model: Qwen/Qwen1.5-1.8B-Chat


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx